## Regional Ocean: Checking Open Boundary Conditions

In [ ]:
import xarray as xr
import os

In [ ]:
CESM_output_dir = ""  # /glade/u/home/manishrv/scratch/archive/" # /glade/derecho/scratch/ajanney/archive/"  "
case_name = ""  # "gustavo-glorys-2"# "CrocCaribGio"

ts_dir = None  # "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing"
base_case_output_dir = None  # None => use CESM_output_dir
base_case_name = None  # "b.e23_alpha17f.BLT1850.ne30_t232.092"

## As regional domains will vary so much in application, we don't want to assume a minimum duration
## Thus, we can ignore start and end dates and simply reduce over the whole time frame.
start_date = None  # "0001-01-01"
end_date = None  # "0101-01-01"
base_start_date = None  # "0001-01-01"
base_end_date = None  # "0101-01-01"

obs_data_dir = "/glade/campaign/collections/rda/data/d010049"  # glorys data

serial = True  # use dask LocalCluster

savefigs = False

lc_kwargs = {}

In [ ]:
OUTDIR = f"{CESM_output_dir}/{case_name}/ocn/hist/"
print("Output directory is:", OUTDIR)

In [ ]:
# When running interactively, cupid_run should be set to 0
# this will spin-up a DASK cluster in the notebook, it may need a few mintues

cupid_run = 1

if cupid_run == 1:

    from dask.distributed import Client, LocalCluster

    # Spin up cluster (if running in parallel)
    client = None
    if not serial:
        cluster = LocalCluster(**lc_kwargs)
        client = Client(cluster)

else:

    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    # Make sure to run on Casper or another system that is able
    # to allocate cores and not just nodes
    cluster = PBSCluster(
        cores=12,
        processes=12,
        memory="120GB",
        account="P93300012",
        queue="casper",
        walltime="02:00:00",
    )

    client = Client(cluster)

    cluster.scale(1)

    print(cluster)

client

In [ ]:
case_output_dir = os.path.join(CESM_output_dir, case_name, "ocn", "hist")

# Xarray time decoding things
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

## Static data includes hgrid, vgrid, bathymetry, land/sea mask
static_data = xr.open_mfdataset(
    os.path.join(case_output_dir, f"*static.nc"),
    decode_timedelta=True,
    decode_times=time_coder,
)

## Surface Data
sfc_data = xr.open_mfdataset(
    os.path.join(case_output_dir, f"*sfc*.nc"),
    decode_timedelta=True,
    decode_times=time_coder,
)

## Monthly Domain Data
monthly_data = xr.open_mfdataset(
    os.path.join(case_output_dir, f"*z*.nc"),
    decode_timedelta=True,
    decode_times=time_coder,
)